# Imports

In [ ]:
import pandas as pd
from hbt_tools import utils_hbt as uh
from hbt_tools import plots as hp
from pathlib import Path
from pytz import timezone

# Functions

In [ ]:
def poleni(h, B, mu=0.5):
    '''
    h: ts [cm]
    B: Breite [m]
    return: Q_pol [l/s]
    '''
    h = h.copy() / 100 # cm -> m
    Q_pol = 2/3 * mu * (2 * 9.81)**(1/2) * B * (h)**(3/2)
    Q_pol = Q_pol * 1e3 # m3/s -> l/s
    return Q_pol
#-------------------------------------------------------------------------------

def sum_per_period(periods, ts, date_format, idx_col=0):
    df_res = pd.DataFrame(columns=['von', 'bis', 'summe'])
    tz = timezone('Europe/Zurich')
    for i, p in enumerate(periods):
        date_start = pd.to_datetime(p[0], format=date_format)
        date_start = tz.localize(date_start)

        date_end = pd.to_datetime(p[1], format=date_format)
        date_end = tz.localize(date_end)

        ts_period = ts[(ts.index>=date_start) & (ts.index<=date_end)]
        df_temp = pd.DataFrame({'von': [p[0]], 'bis': [p[1]],
            'summe': [ts_period.iloc[:,idx_col].sum()]})
        
        df_res = pd.concat((df_res, df_temp))
    
    return df_res
#-------------------------------------------------------------------------------

def ts_filter_date(ts, date_from, date_to):
    ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

def ts_add_vol(ts):
    '''
    ts: timeseries [l/s] or [m3/s]
    return: ts
    '''
    ts['delta_t'] = ts.index
    delta_t = ts['delta_t'] - ts['delta_t'].shift()

    ts['delta_t'] = delta_t
    ts['volumen'] = ts.iloc[:,0] * ts['delta_t'].dt.total_seconds()

    return ts
#-------------------------------------------------------------------------------

def ts_del_values(ts, val, idx_col=0):
    mask = ts.iloc[:,idx_col]==val
    ts_out = ts.drop(ts[mask].index)
    return ts_out
#-------------------------------------------------------------------------------

# Parameters

In [ ]:
date_format = '%d.%m.%Y %H:%M'

tz = timezone('Europe/Zurich')

date_start = pd.to_datetime('01.10.2022 00:00', format=date_format)
date_start = tz.localize(date_start)

date_end = pd.to_datetime('01.10.2023 00:00', format=date_format)
date_end = tz.localize(date_end)

ezg_tot = 2.97 # [ha]
ab_bw = 0.8 # [-]

save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Hydraulischer Wirkungsgrad")

periods = [
    ('28.09.2022 19:49', '08.10.2022 20:49'),
    ('08.10.2022 20:49', '04.11.2022 02:18'),
    ('04.11.2022 12:33', '05.11.2022 12:13'),
    ('05.11.2022 12:13', '10.12.2022 06:44'),
    ('10.12.2022 06:44', '24.12.2022 03:13'),
    ('24.12.2022 03:13', '26.12.2022 19:20'),
    ('26.01.2023 13:39', '12.03.2023 03:31'),
    ('12.03.2023 03:31', '25.03.2023 01:04'),
    ('25.03.2023 01:04', '03.04.2023 14:47'),
    ('03.04.2023 15:04', '24.04.2023 05:35'),
    ('24.04.2023 05:35', '01.05.2023 21:06'),
    ('01.05.2023 21:06', '10.05.2023 03:28'),
    ('31.05.2023 13:50', '19.06.2023 16:53'),
    ('19.06.2023 16:53', '13.07.2023 00:05'),
    ('13.07.2023 00:05', '26.07.2023 01:31'),
    ('26.07.2023 01:31', '28.08.2023 04:54'),
    ('28.08.2023 04:54', '03.10.2023 10:25'),

]

date_from = periods[0][0]
date_to = periods[-1][-1]

# Regen

In [ ]:
id = 2039
ts_rain = uh.hbtdb_get_data(id, date_start, date_end, date_format)

In [ ]:
df_rain_periods = sum_per_period(periods, ts_rain, date_format)
df_rain_periods

In [ ]:
tot_rain = ts_rain.iloc[:,0].sum()
print(f'Tot. Regen: {tot_rain:0.1f} mm')

vol_zu_rain = tot_rain * ezg_tot * 10
print(f'Tot. Zufluss (Regen): {vol_zu_rain:0.2f} m3')

# Zulauf

In [ ]:
id = 465
h_zu = uh.hbtdb_get_data(id, date_start, date_end, date_format)
h_zu = h_zu - 105 # cm, 1.05 Messwert bei ansprigngen des Wehres/Entlastung (Christian)

# Convert Zulauf from h to Q (Poleni)
B = 2.75 # der Wert ist aus dem Schalungsplan
mu = 0.6
q_zu = poleni(B=B, mu=mu, h=h_zu/100) * 1000 # l/s
q_zu.fillna(0, inplace=True)

# Compute volume per time-step
q_zu['delta_t'] = q_zu.index
delta_t = q_zu['delta_t'] - q_zu['delta_t'].shift()
q_zu['delta_t'] = delta_t
q_zu['volumen [l]'] = q_zu['wert'] * q_zu['delta_t'].dt.total_seconds()

In [ ]:
# Check dts
dt_min = delta_t.unique()
dt_min

In [ ]:
df_zufluss = sum_per_period(periods, q_zu, date_format, 2)
df_zufluss.iloc[:,2] /= 1e3 # l -> m3
df_zufluss

In [ ]:
tot_vol_zu = ts_filter_date(q_zu, date_start, date_end)['volumen [l]'].sum() / 1e3
print(f'Zufluss Vol: {tot_vol_zu:0.3f} m3')

# Ablauf

In [ ]:
id = 466
q_ab = uh.hbtdb_get_data(id, date_from, date_to, date_format)
q_ab.index = pd.to_datetime(q_ab.index, utc=True)

# Compute volume per time-step
q_ab['delta_t'] = q_ab.index
delta_t = q_ab['delta_t'] - q_ab['delta_t'].shift()
q_ab['delta_t'] = delta_t
q_ab['volumen'] = q_ab['wert'] * q_ab['delta_t'].dt.total_seconds()

In [ ]:
q_ab['delta_t'].unique()

In [ ]:
df_abfluss = sum_per_period(periods, q_ab, date_format, 2)
df_abfluss.iloc[:,2] /= 1e3 # l -> m3
df_abfluss

In [ ]:
tot_vol_ab = ts_filter_date(q_ab, date_start, date_end)['volumen'].sum() / 1e3
print(f'Abfluss Vol: {tot_vol_ab:0.3f} m3')

# RFB 1

In [ ]:
# RFB 1
id = 459
rfb1 = uh.hbtdb_get_data(id, date_from=date_start, date_to=date_end)
rfb1 = ts_filter_date(rfb1, date_start, date_end)
rfb1 = (rfb1 - 1.73) * -1 * 100 # Correct values, m -> cm
rfb1 = ts_del_values(rfb1, 73)

In [ ]:
# Max Niveau RFB 1
print(f"Max RFB 1: {rfb1.iloc[:,0].max()} cm")
print(f"Datum: {str(rfb1.idxmax().values[0])}")

In [ ]:
h_ül = 60 # cm - gemäss Fotos Begehung
B = 1.5 # m

ü_rfb1 = rfb1 - h_ül
ü_rfb1[ü_rfb1<0] = 0

q_rfb1 = poleni(ü_rfb1, B)
q_rfb1[q_rfb1==0.0] = float('nan')

In [ ]:
fig = hp.ply_1y(q_rfb1, names=['Entlastungsmenge RFB 1'], xlabel='Zeit',
    ylabel='Abfluss [l/s]')
fig

In [ ]:
save_path = save_dir.parent / 'Plots/Entlastungsmengen_RFB1.html'
fig.write_html(save_path)

In [ ]:
# Mengen berechnen, 0-Werte löschen
q_rfb1[q_rfb1.iloc[:,0].isna()] = 0.0
q_rfb1 = ts_add_vol(q_rfb1)
q_rfb1 = ts_del_values(q_rfb1, 0.0, idx_col=2)
q_rfb1.drop(labels=q_rfb1.index[0], inplace=True)
q_rfb1['volumen kumulativ [m3]'] = (q_rfb1['volumen'] / 1000).cumsum()

save_path = save_dir / 'RFB1_Überlaufmengen.csv'
q_rfb1.to_csv(save_path, sep=';')

# RFB 2

In [ ]:
# RFB 2
id = 461
rfb2 = uh.hbtdb_get_data(id, date_from=date_start, date_to=date_end)
rfb2 = ts_filter_date(rfb2, date_start, date_end)
rfb2 = (rfb2 - 0.92) * -1 * 100 # Correct values, m -> cm
rfb2 = ts_del_values(rfb2, 92)

In [ ]:
# Max Niveau RFB 2
print(f" Max RFB 2: {rfb2.iloc[:,0].max():0.1f} cm")
print(f"Datum: {str(rfb2.idxmax().values[0])}")

In [ ]:
h_ül = 67 # cm - gemäss Fotos Begehung
B = 2.1 # m

ü_rfb2 = rfb2 - h_ül
ü_rfb2[ü_rfb2<0] = 0

q_rfb2 = poleni(ü_rfb2, B)
q_rfb2[q_rfb2==0.0] = float('nan')

In [ ]:
fig = hp.ply_1y(q_rfb2, names=['Entlastungsmenge RFB 2'], xlabel='Zeit',
    ylabel='Abfluss [l/s]');

In [ ]:
save_path = save_dir.parent / 'Plots/Entlastungsmengen_RFB2.html'
fig.write_html(save_path)

In [ ]:
# Mengen berechnen, 0-Werte löschen
q_rfb2[q_rfb2.iloc[:,0].isna()] = 0.0
q_rfb2 = ts_add_vol(q_rfb2)
q_rfb2 = ts_del_values(q_rfb2, 0.0, idx_col=2)

save_path = save_dir / 'RFB2_Überlaufmengen.csv'
q_rfb2.to_csv(save_path, sep=';')